In [ ]:
from proj1_helpers import *
import itertools
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

## Load Data

In [ ]:
(y_train, tx_train_raw, ids_train) = (np.array(x) for x in load_csv_data("data/train.csv"))
(y_test, tx_test_raw, ids_test) = (np.array(x) for x in load_csv_data("data/test.csv"))

## Clean Data

In [ ]:
def remove_noisy_column(x):
    """Remove columns for which there are more -999 entries than normal entries"""
    return np.array([i for i in x.T if (i == -999).sum() < (i != -999).sum()]).copy()

In [ ]:
def replace_noise_entries(x):
    """Replace -999 by mean value of the column (mean computed without any -999)"""
    col_means = [np.mean(col[col!= -999]) for col in x.T]
    x_local = x.copy()
    for i,col in enumerate(x_local.T):
        col[col == -999] = col_means[i]
    return x_local

In [ ]:
def get_outliers(x, mean, std):
    outlier_indices = np.zeros() 
    for feature in range(tx.shape[1]):
        row_indices = np.where(np.absolute(tx[:,feature]-mean[feature]) > 3*std[feature])[0]
        mask = np.in1d(row_indices, outlier_indices)
        outlier_indices = np.hstack((outlier_indices, row_indices[np.where(~mask)[0]]))
    
    return outlier_indices.astype(int)

## Test accuracy

In [ ]:
def test_accuracy(w, x_test, y_test):
    """returns accuracy for a specific weight vector"""
    predictions = predict_labels(w, x_test)
    num_equal = (predictions == y_test).sum()
    return num_equals/y_test.shape[0]

## Feature augmentation

In [ ]:
def augment(x, num_important, degree):

    for i, line in enumerate(x):
        ones = np.array([1])
#         for val in itertools.combinations(line[:num_important], r=degree):
#             print(val)
#         comb = np.array([np.prod(val) for val in itertools.combinations(line[:num_important], r=degree)])
        not_imp = np.array([line**d for d in range(2, degree+1)])
        complete = np.concatenate((ones, line, comb, not_imp), axis=None)
        print(ones, line, comb, not_imp, complete)
        if i == 0:
            new = np.zeros((x.shape[0], complete.shape[0]))
        new[i] = complete
    return new

In [ ]:
t = np.array([[1,2,3,4],[2,2,2,2],[5,5,5,5], [3,2,3,2], [2,2,4,4]])
# np.hstack()
augment(t, 3, 3)
# t2 = np.array([1,2,3,4,5])
# print(np.power(t2, 2))


In [ ]:
def MSE(y, tx, w):
    return np.sum(np.power(y - np.dot(tx, w), 2)/(2*len(y)))  # MSE


def MAE(y, tx, w):
    return np.sum(np.abs(y - np.dot(tx, w)))/len(y)  # MAE


def RMSE(y, tx, w):
    return np.sqrt(2*MSE(y, tx, w))


def compute_gradient(y, tx, w):
    """Compute the gradient."""
    e = y - tx.dot(w)
    grad = -tx.T.dot(e)/len(y)
    return grad


def calculate_gradient_log(y, tx, w):
    """compute the gradient of loss."""
    pred = sigmoid(tx.dot(w))
    grad = tx.T.dot(pred - y)
    return grad


def sigmoid(t):
    """apply sigmoid function on t."""
    return 1/(1+np.exp(-t))


def NLL(y, tx, w):
    """compute the cost by negative log likelihood."""
    pred = sigmoid(tx.dot(w))
    loss = y.T.dot(np.log(pred)) + (1 - y).T.dot(np.log(1 - pred))
    return np.squeeze(- loss)
########################
###### ASSIGNMENT ######
########################


def least_squares_GD(y, tx, initial_w, max_iters, gamma):
    w = initial_w
    for n_iter in range(max_iters):
        grad = compute_gradient(y, tx, w)
        loss = MSE(y, tx, w)
        if n_iter % 100 == 0:
            print(loss)
        w = w - gamma*grad
        # print("Step {}, loss is   {}".format(n_iter, loss))
    return (w, loss)


def least_squares_SGD(y, tx, initial_w, max_iters, gamma):
    weights = initial_w
    for _ in range(max_iters):
        rand_index = np.random.randint(y.shape)
        y_batch, tx_batch = y[rand_index], tx[rand_index]
        grad = compute_gradient(y_batch, tx_batch, weights)
        weights = weights - gamma*grad
    loss = MSE(y, tx, weights)
    return (weights, loss)


def least_squares(y, tx):
    a = tx.T.dot(tx)
    b = tx.T.dot(y)
    w = np.linalg.solve(a, b)
    loss = MSE(y, tx, w)
    return (w, loss)


def ridge_regression(y, tx, lambda_):
    aI = 2 * tx.shape[0] * lambda_ * np.identity(tx.shape[1])
    a = tx.T.dot(tx) + aI
    b = tx.T.dot(y)
    w = np.linalg.solve(a, b)
    loss = MSE(y, tx, w)
    return (w, loss)


def reg_logistic_regression(y, tx, lambda_, initial_w, max_iters, gamma):
    pass    


In [ ]:
def logistic_regression(y, tx, initial_w, max_iters, gamma):
    def sigmoid(t):
        """apply sigmoid function on t."""
        return 1/(1+np.exp(-t))

    def calculate_gradient(gradient_y, gradient_tx, gradient_w):
        """compute the gradient of loss."""
        pred = sigmoid(gradient_tx.dot(gradient_w))
        grad = gradient_tx.T.dot(pred - gradient_y)
        return grad

    def calculate_loss(y, loss_tx, loss_w):
        """compute the cost by negative log likelihood."""
        pred = sigmoid(loss_tx.dot(loss_w))
        loss = y.T.dot(np.log(pred)) + (1 - y).T.dot(np.log(1 - pred))
        return np.squeeze(- loss)

    weights = initial_w
    for _ in range(max_iters):
        # get loss and update w.
        loss = calculate_loss(y, tx, weights)
        print(loss)
        grad = calculate_gradient(y, tx, weights)
        weights = weights - gamma * grad
    return (weights, loss)

# MAIN RUN

In [ ]:
from proj1_helpers import *
import itertools
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

In [ ]:
(y, tx, ids) = (np.array(x) for x in load_csv_data("data/train.csv"))

We quickly saw that the data is split into 4 categories, according to the `PRI_jet_num` column. We split according to these categories

In [ ]:
CAT_COL = 22
NUM_CATEGORIES = 4
rows_per_cat = np.array([np.where(tx[:,CAT_COL] == c)[0] for c in range(NUM_CATEGORIES)])

In [ ]:
    # this represents, for each category, which column contain at least one unknown (-999) value
    unknown_cols = [set(np.where(tx[np.where(tx[:, CAT_COL] == c)[0], :] == -999)[1]) for c in range(NUM_CATEGORIES)]

In [ ]:
for i, (rows, cols) in enumerate(zip(rows_per_cat, unknown_cols)):
    percentages = np.asarray([len(np.where(tx[rows, i] == -999)[0]) \
                             /len(rows) for i in cols]) * 100
    print("\nCATEGORY {}:".format(i))
    for col, perc in zip(cols, percentages):
        print("Col {} has {}% of unknown".format(col, perc))

Thus, for every category, any column (except 0) that has any unknown value has only unknown values. This can be explained by the fact that some fields might not be relevant to a certain category, and thus filled with "NaN", or -999

From now on, for all categories, we only keep columns with values different that -999

In [ ]:
for cat in unknown_cols:
    cat.remove(0)

We remove columns PHI, as they are identically distributed and provide no real meaning.
We also remove row 22, as data is now split according to it.
As categories 2 and 3 are basically the same, we consider them as identical

In [ ]:
phi_cols = [15, 18, 20, 25, 28]
columns_to_remove = [np.unique(np.concatenate((list(unknown), [CAT_COL], phi_cols))) for unknown in unknown_cols]
columns_to_remove = columns_to_remove[:-1]
NUM_CATEGORIES = 3
rows_per_cat[2] = np.unique(np.concatenate((rows_per_cat[2],rows_per_cat[3])))
rows_per_cat = rows_per_cat[:-1]

In [ ]:
tx_by_cat = [tx[rows] for rows in rows_per_cat]
y_by_cat = [y[rows] for rows in rows_per_cat]

## Data cleaning

In [ ]:
tx_by_cat = [np.delete(tx_by_cat[cat], np.array(columns_to_remove[cat], dtype=int), axis=1) for cat in range(NUM_CATEGORIES)]

In [ ]:
# Replacing remaining NaN by the average value of the column for the category
for cat in tx_by_cat:
    first_col = cat[:,0]
    first_col[first_col == -999] = np.mean(first_col[first_col != -999])

## Features engineering

In [ ]:
from implementations import *

In [ ]:
from top_features import top_features

In [ ]:
tx_by_cat[0][:,-1]

In [ ]:
top_feat_by_cat, other_feat_by_cat = top_features(tx_by_cat,y_by_cat, 5)

In [17]:
import itertools

import numpy as np

from implementations import *
from proj1_helpers import *
from top_features import top_features

#load the data
(y, tx, ids) = (np.array(x) for x in load_csv_data("data/train.csv"))

#We quickly saw that the data is split into 4 categories, 
# according to the `PRI_jet_num` column. 
# We split according to these categories

CAT_COL = 22
NUM_CATEGORIES = 4
rows_per_cat = np.array([np.where(tx[:,CAT_COL] == c)[0] for c in range(NUM_CATEGORIES)])

# this represents, for each category, which column contain at least one unknown (-999) value
unknown_cols = [set(np.where(tx[np.where(tx[:, CAT_COL] == c)[0], :] == -999)[1]) for c in range(NUM_CATEGORIES)]

# for i, (rows, cols) in enumerate(zip(rows_per_cat, unknown_cols)):
#     percentages = np.asarray([len(np.where(tx[rows, i] == -999)[0]) \
#                              /len(rows) for i in cols]) * 100
#     print("\nCATEGORY {}:".format(i))
#     for col, perc in zip(cols, percentages):
#         print("Col {} has {}% of unknown".format(col, perc))


# Thus, for every category, any column (except 0) that has any unknown value has only unknown values. 
# This can be explained by the fact that some fields might not be relevant to a certain category, and thus filled with "NaN", or -999
# From now on, for all categories, we only keep columns with values different that -999

# remove col 0 from columns with unknown values.
# We'll deal with that later
for cat in unknown_cols:
    cat.remove(0)

#We remove columns PHI, as they are identically distributed and provide no real meaning.
# We also remove row 22, as data is now split according to it.
# As categories 2 and 3 are basically the same, we consider them as identical

phi_cols = [15, 18, 20, 25, 28]
columns_to_remove = [np.unique(np.concatenate((list(unknown), [CAT_COL], phi_cols))) for unknown in unknown_cols]
columns_to_remove = columns_to_remove[:-1]

NUM_CATEGORIES = 3
rows_per_cat[2] = np.unique(np.concatenate((rows_per_cat[2],rows_per_cat[3])))
rows_per_cat = rows_per_cat[:-1]


## Data cleaning
tx_by_cat = [tx[rows] for rows in rows_per_cat]
y_by_cat = [y[rows] for rows in rows_per_cat]

tx_by_cat = [np.delete(tx_by_cat[cat], np.array(columns_to_remove[cat], dtype=int), axis=1) for cat in range(NUM_CATEGORIES)]

# Replacing remaining NaN by the average value of the column for the category
for cat in tx_by_cat:
    first_col = cat[:,0]
    first_col[first_col == -999] = np.mean(first_col[first_col != -999])

top_feat_by_cat, other_feat_by_cat = top_features(tx_by_cat,y_by_cat, 5)

degrees = [7,10,10]
augmented_tx_by_cat = []
print("Computing features augmentation")
for cat, deg, top_feat, other_feat in zip(tx_by_cat, degrees, top_feat_by_cat, other_feat_by_cat):
    augmented_tx_by_cat.append(build_multinomial(cat, deg, top_feat, other_feat))
    
lambdas = [0.001, 0.003, 0.01]
print("Computing weights")
# weights, _ = apply_ridge_regression(y_by_cat, tx_by_cat, degrees, lambdas, 4, 1, top_feat_by_cat, other_feat_by_cat)
weights_by_cat = []
for lamb, tx_cat, y_cat in zip(lambdas, augmented_tx_by_cat, y_by_cat):
    weights_by_cat.append(ridge_regression(y_cat, tx_cat, lamb))

print("Loading test data")
(y_test, tx_test, ids_test) = (np.array(x) for x in load_csv_data("data/test.csv"))

print("Computing predictions")
rows_per_cat_test = [
    np.array([np.where(tx[:,CAT_COL] == 0)[0]]),
    np.array([np.where(tx[:,CAT_COL] == 1)[0]]),
    np.array([np.where(tx[:,CAT_COL] >= 2)[0]])
]

y_predict = np.zeros(tx_test.shape[0])

category_col = tx_test[:,CAT_COL]
#Merge categories 2 and 3 into one single category (2)
cat3_idx = np.where(category_col == 3)
category_col[cat3_idx] = 2*np.ones(len(cat3_idx))


Top features for category 0: [1 7 4 9 2]
Top features for category 1: [0 2 1 8 9]
Top features for category 2: [0 2 4 6 5]
Computing features augmentation
Building multinomial feature matrix
Multinomial partition for (2,5)
Multinomial partition for (3,5)
Multinomial partition for (4,5)
Multinomial partition for (5,5)
Multinomial partition for (6,5)
Multinomial partition for (7,5)
Building multinomial feature matrix
Multinomial partition for (2,5)
Multinomial partition for (3,5)
Multinomial partition for (4,5)
Multinomial partition for (5,5)
Multinomial partition for (6,5)
Multinomial partition for (7,5)
Multinomial partition for (8,5)
Multinomial partition for (9,5)
Multinomial partition for (10,5)
Building multinomial feature matrix
Multinomial partition for (2,5)
Multinomial partition for (3,5)
Multinomial partition for (4,5)
Multinomial partition for (5,5)
Multinomial partition for (6,5)
Multinomial partition for (7,5)
Multinomial partition for (8,5)
Multinomial partition for (9,5)


In [31]:
for i in range(len(weights_by_cat)):
    y_predict[np.where(category_col == i)[0]] =  predict_labels(weights_by_cat[i][0], augmented_tx_by_cat[i])
# [weights_by_cat[i][0] for i in range(3)]

# create_csv_submission(ids_test, y_predict, 'submission.csv')

ValueError: shape mismatch: value array of shape (99913,) could not be broadcast to indexing result of shape (227458,)

In [38]:
category_col.shape

(568238,)